In [2]:
import time
import random
import os
import re
import pandas as pd
import numpy as np
from datetime import datetime
from elasticsearch import Elasticsearch 

In [2]:
claim_df = pd.read_pickle('../data/patent_data/processed/claim_df_0619.pkl')

In [3]:
detail_df = pd.read_pickle('../data/patent_data/processed/detail_sql_df_0619.pkl')

In [4]:
index_df = claim_df[['pub_id','text']].copy()

In [5]:
index_df = pd.merge(left=index_df,right=detail_df,on='pub_id',how='left')

In [6]:
index_df = index_df.reset_index(drop=True)

In [57]:
index_df = index_df.fillna('')

# 筛选数据 当申请日期与发表日期为空时筛掉

In [61]:
index_df = index_df[~((index_df['applicationDate'] == '') & (index_df['publicationDate'] == ''))]

# 部分日期处理

In [63]:
target_date = ['applicationDate','publicationDate']

In [64]:
def date_convert(dot_date_str):
    '''xxxx.xx.xx --> xxxx-xx-xx
    then pack into json for es
    '''
    y,m,d = dot_date_str.split('.')
    new_date = '{year}-{month}-{date}'.format(year=y,month=m,date=d)
    return new_date
#     return {new_date:{'type':'date','formate':'yyyy-mm-dd'}}

In [65]:
for k in target_date:
    index_df[k] = index_df[k].apply(lambda x:date_convert(x))

In [66]:
index_df = index_df.reset_index(drop=True)

# 构建索引

In [ ]:
es = Elasticsearch(hosts='10.8.128.205',port=29200,timeout=30, max_retries=10, retry_on_timeout=True)

# 单条插入

In [67]:
# index_keys = index_df.columns.tolist()
# target_date = ['applicationDate','publicationDate']
# for num,_ in enumerate(index_df.itertuples()):
#     if num %1000==0:
#         print('[{0}] Num:{1}'.format(time.ctime(),num))
#     sample_item = index_df.iloc[num]
#     sample_dict = {index_keys[pnum]:i for pnum,i in enumerate(sample_item)}

#     es.index(index='patent-index',doc_type='patent',id=num,body=sample_dict)
    
# es.indices.refresh(index="patent-index")

# 批量插入

In [12]:
from elasticsearch.helpers import bulk

In [17]:
index_df = index_df.fillna('')

In [69]:
index_keys = index_df.columns.tolist()
counter = 0
index_batch = []
for num,_ in enumerate(index_df.itertuples()):
    if num %5000==0:
        print('[{0}] Num:{1}'.format(time.ctime(),num))
    sample_item = index_df.iloc[num]
    sample_dict = {index_keys[pnum]:i for pnum,i in enumerate(sample_item)}
    if counter <5000:
        index_batch.append(sample_dict)
        counter += 1
    else:
        index_batch.append(sample_dict)
        print('Bulking....')
        time1 = time.time()
        bulk(es,index_batch,index='patent-index',raise_on_error=True)
        time2 = time.time()
        print('Total time:{0:.2f}min'.format((time2-time1)/60))
        counter = 0
        index_batch = []
        
bulk(es,index_batch,index='patent-index',raise_on_error=True)
es.indices.refresh(index="patent-index")

[Thu Jul  4 16:40:15 2019] Num:0
[Thu Jul  4 16:40:16 2019] Num:5000
Bulking....
Total time:0.09min
[Thu Jul  4 16:40:23 2019] Num:10000
Bulking....
Total time:0.09min
[Thu Jul  4 16:40:30 2019] Num:15000
Bulking....
Total time:0.09min
[Thu Jul  4 16:40:36 2019] Num:20000
Bulking....
Total time:0.13min
[Thu Jul  4 16:40:44 2019] Num:25000
Bulking....
Total time:0.09min
[Thu Jul  4 16:40:51 2019] Num:30000
Bulking....
Total time:0.09min
[Thu Jul  4 16:40:57 2019] Num:35000
Bulking....
Total time:0.13min
[Thu Jul  4 16:41:06 2019] Num:40000
Bulking....
Total time:0.09min
[Thu Jul  4 16:41:12 2019] Num:45000
Bulking....
Total time:0.10min
[Thu Jul  4 16:41:19 2019] Num:50000
Bulking....
Total time:0.09min
[Thu Jul  4 16:41:25 2019] Num:55000
Bulking....
Total time:0.12min
[Thu Jul  4 16:41:34 2019] Num:60000
Bulking....
Total time:0.10min
[Thu Jul  4 16:41:41 2019] Num:65000
Bulking....
Total time:0.09min
[Thu Jul  4 16:41:47 2019] Num:70000
Bulking....
Total time:0.13min
[Thu Jul  4 16:4

{'_shards': {'total': 2, 'successful': 1, 'failed': 0}}

In [99]:
query_dict = {
  "query": {
    "bool": {
        "must":[{
      "match": {"text": "药品"}}],
      "filter": {
          "range":{"applicationDate": {
            "gte": "2008-01-01",
            "lte": "2008-12-25"
          }}
        }
      }
    }
}

In [100]:
res = es.search(index="patent-index",body=query_dict)

In [101]:
res

{'took': 94,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': 11.298906,
  'hits': [{'_index': 'patent-index',
    '_type': '_doc',
    '_id': 'SGgqvGsBnjfuaet_nNzm',
    '_score': 11.298906,
    '_source': {'pub_id': 'CN101765771B',
     'text': '0001.1.一种二硫代氨基甲酸盐类农药的分析方法，其特征在于：0001.在反应容器中加入生药或含有生药的制剂或者它们的处理物、L-半胱氨酸或其盐、和提取用有机溶剂，密闭后照射微波，使存在于生药或含有生药的制剂中的二硫代氨基甲酸盐类农药分解为二硫化碳，在所述有机溶剂中分配提取之后进行分析。0002.2.如权利要求1所述的方法，其特征在于：0002.进一步在反应容器中加入酸催化剂水溶液并密闭。0003.3.如权利要求2所述的方法，其特征在于：0003.酸催化剂水溶液含有氯化锡(II)或其水合物和盐酸。0004.4.如权利要求1～3中任一项所述的方法，其特征在于：0004.提取用有机溶剂是环己烷或2，2，4-三甲基戊烷。0005.5.如权利要求1所述的方法，其特征在于：0005.利用气相色谱进行分析。0006.6.如权利要求2所述的方法，其特征在于：0006.利用气相色谱进行分析。0007.7.如权利要求3所述的方法，其特征在于：0007.利用气相色谱进行分析。0008.8.如权利要求4所述的方法，其特征在于：0008.利用气相色谱进行分析。0009.9.如权利要求1所述的方法，其特征在于：0009.二硫代氨基甲酸盐类农药包括丙森锌。0010.10.如权利要求2所述的方法，其特征在于：0010.二硫代氨基甲酸盐类农药包括丙森锌。0011.11.如权利要求3所述的方法，其特征在于：0011.二硫代氨基甲酸盐类农药包括丙森锌。0012.12.如权利要求4所述的方法，

In [51]:
res = es.search(index="test-index", body={"query": {"match_all": {}}})
print("Got %d Hits:" % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print("%(timestamp)s %(author)s: %(text)s" % hit["_source"])

Got 1 Hits:
2019-07-03T10:37:12.359025 kimchy: Elasticsearch: cool. bonsai cool.


In [3]:
class RetirevalMeta:
    '''检索系统父类 定义三种方法
    init部分负责检索服务器连接
    do_search执行检索过程
    format_search对检索结果进行格式化后处理
    Retrieval将上述两种方法结合 返回最终结果
    '''
    def __init__(self,):
        pass
    def do_search(self,):
        pass
    def format_search(self,):
        pass
    def Retrieval(self,):
        pass

In [37]:
class esPatentRetrieval(RetirevalMeta):
    def __init__(self,host,port):
        '''
        使用ES进行专利检索 指定host、port以及专利index之后进行检索
        '''
        super(esPatentRetrieval, self).__init__()
        self.es = Elasticsearch(hosts=host,port=port,timeout=30, max_retries=10, retry_on_timeout=True)
        self.indexName = 'patent-index'
        
    def do_search(self,query,fromDate,toDate,volume):
        '''
        do_search方法执行具体检索过程
        query传入检索词 formDate和toDate分别为检索的前后时间
        volume为返回文档数目
        '''
        queryKeywordDict = {"match": {"text": query}
                }
        queryDateRangeDict = {"range": {
                    "publicationDate": {
                      "gt": fromDate,
                      "lt": toDate
                    }
                  }
                }
        queryBody = {"query": {"bool": {
              "must": [queryDateRangeDict,queryKeywordDict],
              "must_not": [],
              "should": []
            }},
          "from": 0,
          "size": volume,
          "sort": [],
          "aggs": {}
        }
        result = es.search(index=self.indexName,body=queryBody)
        return result
    def format_search(self,result):
        '''
        format_search方法对检索结果进行格式化 构建符合要求的字段进行返回
        输入result为检索结果 提取其中的检索结果进行后处理
        使用ES检索后得到的结果中result['hits']['hits']为数组格式数据
        其中每一个元素为一个dict 对应部分字段
        '''
        docs = result['hits']['hits']
        docs = [i['_source'] for i in docs]
        targetKeyList = 'id pub_id abstractDesc text examiner priorityDocNum agentName title applicationDocNum agentPersonName applicant ipcMain priorityDate inventor assignee publicationDate ipcList applicationDate'
        targetKeyList = targetKeyList.split(' ')

        dict_filter_by_keys = lambda d:{k:d[k] for k in targetKeyList}
        dict_filter_text = lambda d:{k if not k=='text' else 'claim_text':d[k] for k in d}
        dict_filter_id = lambda d:{k if not k=='_id' else 'id':d[k] for k in d}
        
        
        docs = (dict_filter_by_keys(doc) for doc in docs)
        docs = [dict_filter_text(doc) for doc in docs]
        return docs
    
    def Retrieval(self,query,fromDate,toDate,volume):
        result = self.do_search(query,fromDate,toDate,volume)
        docs = self.format_search(result)
        return docs

In [38]:
ess = esPatentRetrieval(host='127.0.0.1',port=9200)

In [ ]:
ess.Retrieval('机器人','2000-01-22','2010-01-22',volume=100)